In [1]:
import json
import sys
import os
import numpy as np
import oursin as urchin

from one.api import ONE
one = ONE()
one.alyx.clear_rest_cache()
one = ONE()
import ibllib.atlas as atlas
ba = atlas.AllenAtlas(25)




C:\Users\noamroth\anaconda3\envs\iblenv\lib\site-packages\ibllib\atlas\regions.py:439: RuntimeWarning: invalid value encountered in cast
  level=df_regions.depth.to_numpy().astype(np.uint16),
C:\Users\noamroth\anaconda3\envs\iblenv\lib\site-packages\ibllib\atlas\regions.py:441: RuntimeWarning: invalid value encountered in cast
  order=df_regions.graph_order.to_numpy().astype(np.uint16))


In [2]:
urchin.setup()

(URN) connected to server
Login sent with ID: 445e9f27, copy this ID into the renderer to connect.


In [32]:
urchin.ccf25.load()

(Warning) Atlas was already loaded, the renderer can have issues if you try to load an atlas twice.


In [4]:
#change subject name here:
subject = 'NR_0028'
traj_subj_micro = one.alyx.rest('trajectories', 'list', subject=subject, provenance = 'Micro-manipulator')

print('Number of micromanipulator trajectories in this mouse: ', len(traj_subj_micro))

Number of micromanipulator trajectories in this mouse:  7


In [35]:
#Loop through all of the trajectories, get the entry/tip coordinates and angles, and plot
import random
no_of_colors=len(traj_subj_micro)
color=["#"+''.join([random.choice('0123456789ABCDEF') for i in range(6)])
       for j in range(no_of_colors)]

count = 0
probe_list = urchin.probes.create(len(traj_subj_micro)) #create a list of probes with length trajectories
probe_names = []
session_numbers = []
probe_entry_coords = []
probe_depth_coords = []
probe_angle_coords = []
for traj_micro in traj_subj_micro:

    if len(traj_micro)>0: # skip empty trajectories, so these will just be blank in the probe_list
    
        ins_micro = atlas.Insertion.from_dict(traj_micro)
        probe_name = traj_micro['session']['start_time'][0:10] + '  ' + traj_micro['probe_name']
        session_number = traj_micro['session']['number']
       
        try:                      
            entry_coords_micro = ba.xyz2ccf(ins_micro.entry, mode='wrap')
            tip_coords_micro = ba.xyz2ccf(ins_micro.tip,mode='wrap')
            tip_coords_micro = [tip_coords_micro[1], tip_coords_micro[0], tip_coords_micro[-1]] #swap X/Y from IBL (X,Y,Z) to be compatible with urchin (AP,ML,DV)
           
            if ins_micro.phi ==15: #this means user error swapped phi and theta
                theta = ins_micro.phi
                phi = ins_micro.theta
                ins_micro.theta = theta
                ins_micro.phi = phi
                
            tip_angles_micro = [ins_micro.phi-90, 90-ins_micro.theta, ins_micro.beta]
            print(tip_angles_micro)
        except Exception as error:
            print('Not able to display this trajectory, pid ={}, {} '.format( traj_micro['probe_insertion'], probe_name))
            print(error)
            continue                
        
        
        probe_list[count].set_position(tip_coords_micro)
        probe_list[count].set_color(color[count])
        probe_list[count].set_angle(tip_angles_micro)
        probe_names.append(probe_name)
        session_numbers.append(session_number)
        probe_entry_coords.append(ins_micro.entry * 1e6)
        probe_depth_coords.append(ins_micro.depth*1e6)
        probe_angle_coords.append(ins_micro.theta)
            
            
        count+=1
    else:
        print('Empty trajectory')
        continue

full_probe_details = [probe_names[i] + '  00'+ str(session_numbers[i]) + '  ' + str(probe_entry_coords[i]) + ' ' +  str(probe_depth_coords[i]) + 
                      '  ' + str(probe_angle_coords[i]) for i in range(len(probe_names))]


[90.0, 75.0, 0]
[90.0, 75.0, 0]
[90.0, 75.0, 0]
[-90.0, 75.0, 0]
[90.0, 75.0, 0]
[-90.0, 75.0, 0]
[90.0, 75.0, 0]


In [34]:

urchin.ccf25.grey.set_visibility(True)
urchin.ccf25.grey.set_material("transparent-unlit")
urchin.ccf25.grey.set_alpha(0.3)

#Creating legend (text to list probe names, session numbers, and entry/angle/depth) 
text_list = urchin.text.create(len(traj_subj_micro))
urchin.text.set_texts(text_list, full_probe_details)
y_positions = np.arange(1,1-0.1*len(traj_subj_micro),-.1)

urchin.text.set_positions(text_list, [[-1,y_positions[i]] for i in range(len(y_positions))])
urchin.text.set_font_sizes(text_list, [24]*len(text_list))
urchin.text.set_colors(text_list, color)

In [ ]:
[ins_micro.tip[1], ins_micro.tip[0], ins_micro.tip[-1]]

In [ ]:
ins_micro.tip

In [ ]:
ins_micro

In [12]:
tip_coords_micro

[-1783.0000000000002, 46.91427061512461, -5935.55495773441]

In [14]:
full_probe_details

['2023-03-20  probe00  001  [-1914. -3575.     0.] 6400.0  15.0',
 '2023-03-14  probe00  001  [-1714.4 -1783.1  -251.9] 6316.4  15.0',
 '2023-03-07  probe00  001  [-2243.6 -2000.6  1062.3] 3977.3000000000006  15.0',
 '2023-03-07  probe01  001  [ 2122.  -2000.2  1131.5] 3965.9000000000005  0.0',
 '2023-03-15  probe00  003  [-1506.5 -1783.4  -147.1] 6024.7  15.0',
 '2023-03-15  probe01  003  [-2950.8 -2241.5  1127.8] 4397.0  0.0',
 '2023-03-10  probe00  001  [-1506. -1783.  -140.] 6000.0  15.0']